In [32]:
# Load model directly
from transformers import TrainingArguments, Trainer, AutoTokenizer, DataCollatorForLanguageModeling, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import load_dataset
import pandas as pd
import torch

model = AutoModelForSequenceClassification.from_pretrained("bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12", num_labels=3)
print(model)

# Move model to the available device (cuda or cpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [33]:
# Get the dataset
dataset = load_dataset("qiaojin/PubMedQA", "pqa_labeled")
small_dataset = dataset['train'].select(range(100))

print(small_dataset)

def preprocess(text):
    questions = text['question']
    context = text['context']
    long_answer = text['long_answer']
    final_decision = text['final_decision']

    # Combine the question, context, and long answer
    text = [f"Context: {c} Question: {q}" for c, q in zip(context, questions)]
    token_input = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

    # Labels
    label_mapping = {"yes": 0, "no": 1, "maybe": 2}
    labels = [label_mapping[l] for l in final_decision]

    # Debugging information
    print(f"Processed text: {text}", len(text))
    print(f"Labels: {labels}", len(labels))
    print(f"Tokenized input: {token_input['input_ids']}", len(token_input['input_ids']))

    # Add the labels to the token_input
    token_input['labels'] = torch.tensor(labels, dtype=torch.long)

    return token_input

# Tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained("bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12")

tokenized_input = small_dataset.map(preprocess, batched=True)

print(tokenized_input)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Split the dataset
train_size = 60
validate_size = 20
test_size = 20

train_dataset, validate_dataset, test_dataset = torch.utils.data.random_split(tokenized_input, [train_size, validate_size, test_size])

# Train the model
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    eval_strategy="epoch",     # evaluation strategy to adopt during training
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    learning_rate=2e-5,              # learning rate
    weight_decay=0.01,               # strength of weight decay
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=validate_dataset,        # evaluation dataset
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

# Evaluate the model
results = trainer.evaluate(test_dataset)
print("Test results:", results)

Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
    num_rows: 100
})
Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 100
})


Epoch,Training Loss,Validation Loss
1,No log,1.445872
2,No log,1.733259
3,No log,1.765103


Test results: {'eval_loss': 2.2138173580169678, 'eval_runtime': 0.5662, 'eval_samples_per_second': 35.325, 'eval_steps_per_second': 35.325, 'epoch': 3.0}


In [41]:
def predict_answer(context, question):
    text = f"Context: {context['contexts']} Question: {question}"
    token_input = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512).to(device)

    # Generate model output (logits)
    with torch.no_grad():
        outputs = model(**token_input)

    # Extract the logits from the model output
    logits = outputs.logits

    # Get the predicted class (use argmax to get the index of the highest score)
    predicted_class = torch.argmax(logits, dim=-1).item()

    # Map the index to the label
    label_mapping = {0: "yes", 1: "no", 2: "maybe"}
    prediction = label_mapping[predicted_class]

    return prediction

count = 0
for example in test_dataset:
    context = example['context']
    question = example['question']
    answer = example['final_decision']
    prediction = predict_answer(context, question)

    print(f"Context: {context['contexts']}")
    print()
    print(f"Question: {question}")
    print()
    print(f"Predicted answer: {prediction}")
    print()
    print(f"Actual answer: {answer}")
    print("="*120)

    if prediction == answer:
        count += 1

print(f"Accuracy: {count/len(test_dataset)}")

Context: ['PRISM is claimed to score disease severity which has attributed an impact on length of PICU stay (LOS).', "To determine the impact of PRISM on LOS, and evaluate an Artificial Neural Network's (ANN) performance to estimate LOS from PRISM item patterns.", "Retrospectively we performed correlation and regression analyses on routinely scored PRISM data of all consecutive admissions to our level-III PICU from 1994 to 1999 (n>2000) with individual LOS. In addition, an ANN was trained on the chronologically first 75% of those data (inputs, PRISM items + age + sex; output, LOS). The ANN's performance was tested on the remaining most recent 25% of the data sets.", "The Spearman and Pearson coefficients of correlation between PRISM and LOS were 0.2 (p<0.001) and 0.08 (p = 0.0003), the latter being slightly higher when LOS was logarithmically transformed. Pearson's coefficient of correlation between ANN derived LOS estimate and actual LOS was 0.21 (p<0.001) (LOS logarithmically transfo